 **Clinton Alden**

 **Mountain Hydrology Research Group**

 **University of Washington**

In [2]:
snotel = "672:WA:SNTL" #input('Enter the desired SNOTEL site code (ie. 1107:WA): ') + ':SNTL'
water_year = 2025# int(input('Enter the water year: '))
out_name = "Olallie Meadows" #input('Enter the output file name (ie. buck_WY16): ')
# out_path = input('Enter the output path (ie. ../model/forcings/): ')
out_path = './forcings/'

print('********** generating the forcing file, please be patient **********')
print('********** should take ~3 minutes to run **********')

********** generating the forcing file, please be patient **********
********** should take ~3 minutes to run **********


 ## Use metloom API to pull snotel data

In [4]:
import warnings
# pysumma has many depreciated packages, this ignores their warnings
warnings.filterwarnings("ignore", category=UserWarning, module='scipy')

from datetime import datetime, timedelta
from metloom.pointdata import SnotelPointData
import pandas as pd
import geopandas as gpd
import xarray as xr
from metsim import MetSim
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
from metpy.units import units
import metpy.calc as mpcalc
import math
import scipy
import os
import shutil
from pytz import UTC


import sys
from utils import lw_clr
from utils import forcing_filler as ff
from utils import summa_check as sc



In [5]:
#  Create needed directories to store metsim run and snotel CSVs
if not os.path.exists('./input/'):
    os.makedirs('./input/')

if not os.path.exists('./out/'):
    os.makedirs('./out/')

if not os.path.exists('./snotel_csvs/'):
    os.makedirs('./snotel_csvs/')

if not os.path.exists('./forcings/'):
    os.makedirs('./forcings/')

# metsim and metloom require different formats of time and ranges, reformat here
water_year_str = str(water_year)
start_year = water_year - 1
start_year_str = str(start_year)

start_date = datetime(start_year, 7, 3)
end_date = datetime(water_year, 8, 31)

spinstart = pd.to_datetime(start_year_str + '-07-03').tz_localize('UTC')
spinend = pd.to_datetime(start_year_str + '-09-30').tz_localize('UTC')

start_loc = datetime(start_year, 10, 1).replace(tzinfo=UTC)
mask_date = datetime(start_year, 10, 2).replace(tzinfo=UTC)

if start_year == datetime.now().year:
    dates = pd.date_range(f"{start_year_str}-10-01", datetime.now().strftime('%Y-%m-%d'))
else:
    dates = pd.date_range('10/01/' + start_year_str, '08/31/' + water_year_str)

spin_range = pd.date_range('07/03/' + start_year_str, '09/30/' + start_year_str)


In [6]:
# Pull desired variables from snotel to dataframe
snotel_point = SnotelPointData(snotel, "MyStation")
df = snotel_point.get_hourly_data(
    start_date, end_date,
    [snotel_point.ALLOWED_VARIABLES.PRECIPITATIONACCUM, snotel_point.ALLOWED_VARIABLES.TEMP, 
     snotel_point.ALLOWED_VARIABLES.SWE, snotel_point.ALLOWED_VARIABLES.SNOWDEPTH]
)

# Specify latitude, longitude, and elevation from station metadata
lat = snotel_point.metadata.y
lon = snotel_point.metadata.x
elev = snotel_point.metadata.z

# Clean up the dataframe
df.reset_index(inplace=True)

# Rename columns
replace = {'ACCUMULATED PRECIPITATION':'accppt','AIR TEMP':'airtemp', 'datetime':'time'}
df.rename(columns=replace, inplace=True)
df.set_index('time', inplace=True)

df.to_csv('./snotel_csvs/'+out_name+'.csv')

# add 'SNOWDEPTH' and 'SNOWDEPTH_units' to the droplist if it decides to work again
try:
    df.drop(columns=['site', 'ACCUMULATED PRECIPITATION_units', 'geometry', 'AIR TEMP_units', 'datasource', 
                 'SWE', 'SWE_units', 'SNOWDEPTH', 'SNOWDEPTH_units'], inplace=True)
except:
    df.drop(columns=['site', 'ACCUMULATED PRECIPITATION_units', 'geometry', 'AIR TEMP_units', 'datasource', 
                 'SWE', 'SWE_units'], inplace=True)
    print('SNOTEL csv has no snowdepth for this run')


 ## Fill missing timesteps from snotel data

In [7]:
# Convert the index of the dataframe to a DatetimeIndex
df.index = pd.to_datetime(df.index)

# Create a date range from the first to the last timestep
date_range = pd.date_range(start=df.index.min(), end=df.index.max(), freq='h')

# Find the missing dates
missing_dates = date_range[~date_range.isin(df.index)]

# Reindex the data DataFrame with the missing dates
# Concatenate the original DataFrame with a DataFrame containing the missing dates
new_df = pd.concat([df, pd.DataFrame(index=missing_dates)], axis=0)

# Sort the new DataFrame by the index
new_df = new_df.sort_index()
df = new_df

# Fill NaNs for every other column
df = df.fillna(np.nan)

# Rename index
df.index.name = 'time'


 ## Unit Conversions

In [8]:
# Covert air temperature to celsius
df['airtemp'] = (df['airtemp'] - 32) * 5.0/9.0

# Convert precipitation to mm
df['accppt'] = df['accppt'] * 25.4

# Convert from geodataframe to dataframe
df = pd.DataFrame(df)



 Split up data into spinup state and desired date range for MetSim

In [9]:
# Interpolate the missing values
df.interpolate(inplace=True)

# Seperate the data into two dataframes, before and after October 1
spinup = df.loc[spinstart:spinend].copy()
data = df.loc[start_loc:]

# Copy the dataframe a2 to a2_copy
data_copy = data.copy()

# Create a mask to identify rows where the index is less than or equal to October 2, 2023
mask = data_copy.index <= mask_date

# Set the 'precip_accum' column to 0 for rows that satisfy the mask condition
data_copy.loc[mask, 'accppt'] = 0

# Update the value of a2 to the modified copy
data = data_copy

# Calculate the difference between the maximum value of 'precip_accum' and the previous value
spinup['pptrate'] = spinup['accppt'].cummax().diff()
data['pptrate'] = data['accppt'].cummax().diff()

# Drop accppt column
spinup.drop(columns=['accppt'], inplace=True)
data.drop(columns=['accppt'], inplace=True)


 ## Generate SW from MetSim

In [10]:
# Create empty dataset
shape = (len(dates), 1, 1, )
dims = ('time', 'lat', 'lon', )

# We are running only one site, at these coordinates
lats = [lat]
lons = [lon]
elev = elev # meters
coords = {'time': dates, 'lat': lats, 'lon': lons}

# Create the initial met data input data structure
met_data = xr.Dataset(coords=coords)


In [20]:
for varname in ['prec', 't_min', 't_max']:
    met_data[varname] = xr.DataArray(data=np.full(shape, np.nan),
                                     coords=coords, dims=dims,
                                     name=varname)


In [12]:
met_data

<xarray.Dataset>
Dimensions:  (time: 51, lat: 1, lon: 1)
Coordinates:
  * time     (time) datetime64[ns] 2024-10-01 2024-10-02 ... 2024-11-20
  * lat      (lat) float64 47.37
  * lon      (lon) float64 -121.4
Data variables:
    prec     (time, lat, lon) float64 nan nan nan nan nan ... nan nan nan nan
    t_min    (time, lat, lon) float64 nan nan nan nan nan ... nan nan nan nan
    t_max    (time, lat, lon) float64 nan nan nan nan nan ... nan nan nan nan

In [24]:
# Resample the data to daily frequency and calculate the maximum and minimum temperatures
tmax_vals = data['airtemp'].resample('D').max().to_xarray()
tmax_vals['time'] = pd.to_datetime(tmax_vals.time.values)

tmin_vals = data['airtemp'].resample('D').min().to_xarray()
tmin_vals['time'] = pd.to_datetime(tmin_vals.time.values)

# Calculate the daily precipitation values
prec_vals = data['pptrate'].resample('D').sum().to_xarray()
# convert time to datetime
prec_vals['time'] = pd.to_datetime(prec_vals.time.values)

met_data['prec'] = prec_vals
# Assign the daily maximum and minimum temperatures to the met_data xarray, converting to Celsius
met_data['t_min'] = tmin_vals
met_data['t_max'] = tmax_vals
# met_data.to_netcdf('./input/rc_forcing.nc')


<xarray.Dataset>
Dimensions:  ()
Data variables:
    prec     int64 0
    t_min    int64 0
    t_max    int64 0

In [22]:
# We form the domain in a similar fashion
# First, by creating the data structure
coords = {'lat': lats, 'lon': lons}
domain = xr.Dataset(coords=coords)
domain['elev'] = xr.DataArray(data=np.full((1,1,), np.nan),
                          coords=coords,
                          dims=('lat', 'lon', ))
domain['mask'] = xr.DataArray(data=np.full((1,1,), np.nan),
                          coords=coords,
                          dims=('lat', 'lon', ))

# Add the data
domain['elev'][0, 0] = elev
domain['mask'][0, 0] = 1
domain.to_netcdf('./input/rc_domain.nc')


PermissionError: [Errno 13] Permission denied: b'/home/dlhogan/GitHub/pysumma/src/input/rc_domain.nc'

In [23]:
# Finally, we create the state file - the dates are 90 days prior to 
# the MetSim run dates - as usual, create an empty data structure to
# read the data into
shape = (len(spin_range), 1, 1, )
dims = ('time', 'lat', 'lon', )
coords = {'time': spin_range, 'lat': lats, 'lon': lons}
state = xr.Dataset(coords=coords)
for varname in ['prec', 't_min', 't_max']:
    state[varname] = xr.DataArray(data=np.full(shape, np.nan),
                               coords=coords, dims=dims,
                               name=varname)
    
# Resample precip to daily
prec_vals = spinup['pptrate'].resample('D').sum()

# Resample the data to daily frequency and calculate the maximum and minimum temperatures
tmax_vals = spinup['airtemp'].resample('D').max()
tmin_vals = spinup['airtemp'].resample('D').min()

# Do precip data
state['prec'].values[:, 0, 0] = prec_vals

# And now temp data and convert to C
state['t_min'].values[:, 0, 0] = tmin_vals
state['t_max'].values[:, 0, 0] = tmax_vals
state.to_netcdf('./input/rc_state.nc')


PermissionError: [Errno 13] Permission denied: b'/home/dlhogan/GitHub/pysumma/src/input/rc_state.nc'

In [ ]:
# dates = pd.date_range('10/01/2014', '09/30/2015')
params = {
    'time_step'    : "60",       
    'start'        : dates[0],
    'stop'         : dates[-1],
    'forcing'      : './input/rc_forcing.nc',     
    'domain'       : './input/rc_domain.nc',
    'state'        : './input/rc_state.nc',
    'forcing_fmt'  : 'netcdf',
    'out_dir'      : './out',
    'out_prefix': out_name,
    'scheduler'    : 'threading',
    'chunks'       : 
        {'lat': 1, 'lon': 1},
    'forcing_vars' : 
        {'prec' : 'prec', 't_max': 't_max', 't_min': 't_min'},
    'state_vars'   : 
        {'prec' : 'prec', 't_max': 't_max', 't_min': 't_min'},
    'domain_vars'  : 
        {'elev': 'elev', 'lat': 'lat', 'lon': 'lon', 'mask': 'mask'}
    }               

# Run MetSim
ms = MetSim(params)
ms.run()
output = ms.open_output().load()

# Delete MetSim input and output directories to declutter, they are unnecessary
if os.path.exists('./input/'):
    shutil.rmtree('./input/')

if os.path.exists('./out/'):
    shutil.rmtree('./out/')


 ## Create SUMMA forcing netCDF

In [ ]:
out_df = output.to_dataframe()
out_df.reset_index(inplace=True)
out_df.set_index('time', inplace=True)


In [ ]:
# Remove timezone from index
data.index = data.index.tz_convert(None)

# Convert precipitation rate from m hr^-1 to kg m^-2 s^-1
data['pptrate'] = data['pptrate']/3600

# Generate relative humidity assuming T_d is overnight low temperature
# Used to calculate specific humidity and longwave radiation
ff.fill_rel_hum(data)

# Convert airtemp to Kelvin
data['airtemp'] = (1.03*(data['airtemp']-0.9)) + 273.15 # Currier snotel temp correction

# Generate pressure from hypsometric equation and site elevation (1981m)
ff.fill_pressure(data, elev)

# Generate specific humidity
ff.fill_spec_hum(data)
data['spechum'] = data['spechum'].clip(lower=0.001)


# Set shortwave radiation to MetSim output
data['SWRadAtm'] = out_df['shortwave']

# Generate longwave radiation
data['LWRadAtm'] = lw_clr.dilleyobrien1998(data['airtemp'], data['rh'])

# Can alternatively use the MetSim LW radiation
# data['LWRadAtm'] = out_df['longwave']

# Set wind to 2 m/s
data['windspd'] = 2

# Fill in missing values
data['pptrate'] = data['pptrate'].fillna(0)

# Drop unnecessary columns
data = data.drop(columns=['rh'])

# Interpolate the missing values
data.interpolate(inplace=True)


In [ ]:
# Load template forcing file to preserve attributes
template = xr.open_dataset('./summa_forcing_template.nc')

# Convert dataframe to xarray
dsx = data.to_xarray()

# Loop through variables and add attributes from template forcing file
for data_var in dsx:
    dsx[data_var].attrs = template[data_var].attrs
    
# Add hru dimension
dsx = dsx.expand_dims(dim={'hru':1})

# Add gap-filled and datastep variables
dsx['gap_filled'] = xr.DataArray(np.ones((1,dsx.time.shape[0])),dims = ['hru','time'])
dsx['data_step'] = 3600 # 3600 seconds for 1hr timesteps

# Convert all to float64
for var in dsx.data_vars:
    dsx[var] = dsx[var].astype(np.float64)

# Set hruID based on template
dsx['hruId'] = (xr.DataArray(np.ones((1))*template['hruId'].values,dims = ['hru'])).astype(np.int32)

# Transpose all variables to match SUMMA dimensions
count = 0
for var in dsx.data_vars:
    # print(var,count)
    count += 1
    if count <= 7:
        attribs = dsx[var].attrs
        arr_t = dsx[var].values.T
        dsx[var] = xr.DataArray(dims = ['time','hru'],data = arr_t)
        dsx[var].attrs = attribs

# Set hruID based on template
dsx['hruId'] = (xr.DataArray(np.ones((1))*template['hruId'].values,dims = ['hru'])).astype(np.float64).fillna(0).astype(np.int32)

# Set time to match SUMMA format and save
dsx.to_netcdf(out_path+out_name+'.nc',
                        encoding = {"time":
                                        {'dtype' : 'float64',
                                         'units' : 'hours since 1990-01-01 00:00:00',
                                         'calendar' : 'standard'}})


